In [1]:
import numpy as np
import math
from PIL import Image
import time

# Packages we're using
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage
from tqdm import tqdm

%matplotlib inline
import IPython.display
from ipywidgets import interact, interactive, fixed

# Packages we're using
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage

import glob
import os
from tqdm import tqdm
# plt, plot, tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [3]:
FFT_LENGTH = 512
WINDOW_LENGTH = 512
WINDOW_STEP = int(WINDOW_LENGTH / 2)
phaseMax = 3.141592653589793 
phaseMin = -3.141592653589793
magnitudeMax = 5432292.907520762 #3212111.6378762764 # 5432292.907520762
magnitudeMin = 0.0

In [4]:
def amplifyMagnitudeByLog(d):
    return 188.301 * math.log10(d + 1)

def weakenAmplifiedMagnitude(d):
    return math.pow(10, d/188.301)-1

def generateLinearScale(magnitudePixels, phasePixels, 
                        magnitudeMin, magnitudeMax, phaseMin, phaseMax):
    height = magnitudePixels.shape[0]
    width = magnitudePixels.shape[1]
    magnitudeRange = magnitudeMax - magnitudeMin
    phaseRange = phaseMax - phaseMin
    rgbArray = np.zeros((height, width, 3), 'uint8')
    
    for w in range(width):
        for h in range(height):
            magnitudePixels[h,w] = (magnitudePixels[h,w] - magnitudeMin) / (magnitudeRange) * 255 * 2
            magnitudePixels[h,w] = amplifyMagnitudeByLog(magnitudePixels[h,w])
            phasePixels[h,w] = (phasePixels[h,w] - phaseMin) / (phaseRange) * 255
            red = 255 if magnitudePixels[h,w] > 255 else magnitudePixels[h,w]
            green = (magnitudePixels[h,w] - 255) if magnitudePixels[h,w] > 255 else 0
            blue = phasePixels[h,w]
            rgbArray[h,w,0] = int(red)
            rgbArray[h,w,1] = int(green)
            rgbArray[h,w,2] = int(blue)
    return rgbArray

def recoverLinearScale(rgbArray, magnitudeMin, magnitudeMax, 
                       phaseMin, phaseMax):
    width = rgbArray.shape[1]
    height = rgbArray.shape[0]
    magnitudeVals = rgbArray[:,:,0].astype(float) + rgbArray[:,:,1].astype(float)
    phaseVals = rgbArray[:,:,2].astype(float)
    phaseRange = phaseMax - phaseMin
    magnitudeRange = magnitudeMax - magnitudeMin
    for w in range(width):
        for h in range(height):
            phaseVals[h,w] = (phaseVals[h,w] / 255 * phaseRange) + phaseMin
            magnitudeVals[h,w] = weakenAmplifiedMagnitude(magnitudeVals[h,w])
            magnitudeVals[h,w] = (magnitudeVals[h,w] / (255*2) * magnitudeRange) + magnitudeMin
    return magnitudeVals, phaseVals

In [5]:
def generateSpectrogramForWave(signal):
    start_time = time.time()
    global magnitudeMin
    global magnitudeMax
    global phaseMin
    global phaseMax
    buffer = np.zeros(int(signal.size + WINDOW_STEP - (signal.size % WINDOW_STEP)))
    buffer[0:len(signal)] = signal
    height = int(FFT_LENGTH / 2 + 1)
    width = int(len(buffer) / (WINDOW_STEP) - 1)
    magnitudePixels = np.zeros((height, width))
    phasePixels = np.zeros((height, width))

    for w in range(width):
        buff = np.zeros(FFT_LENGTH)
        stepBuff = buffer[w*WINDOW_STEP:w*WINDOW_STEP + WINDOW_LENGTH]
        # apply hanning window
        stepBuff = stepBuff * np.hanning(WINDOW_LENGTH)
        buff[0:len(stepBuff)] = stepBuff
        #buff now contains windowed signal with step length and padded with zeroes to the end
        fft = np.fft.rfft(buff)
        for h in range(len(fft)):
            #print(buff.shape)
            #return
            magnitude = math.sqrt(fft[h].real**2 + fft[h].imag**2)
            if magnitude > magnitudeMax:
                magnitudeMax = magnitude 
            if magnitude < magnitudeMin:
                magnitudeMin = magnitude 

            phase = math.atan2(fft[h].imag, fft[h].real)
            if phase > phaseMax:
                phaseMax = phase
            if phase < phaseMin:
                phaseMin = phase
            magnitudePixels[height-h-1,w] = magnitude
            phasePixels[height-h-1,w] = phase
    rgbArray = generateLinearScale(magnitudePixels, phasePixels,
                                  magnitudeMin, magnitudeMax, phaseMin, phaseMax)
    
    
    elapsed_time = time.time() - start_time
    print('%.2f' % elapsed_time, 's', sep='')
    img = Image.fromarray(rgbArray, 'RGB')
    return img

In [6]:
def recoverSignalFromSpectrogram(numpyarray):
#     img = Image.open(filePath)
    data = np.array( numpyarray, dtype='uint8' )
    width = data.shape[1]
    height = data.shape[0]

    magnitudeVals, phaseVals \
    = recoverLinearScale(data, magnitudeMin, magnitudeMax, phaseMin, phaseMax)
    
    recovered = np.zeros(WINDOW_LENGTH * width // 2 + WINDOW_STEP, dtype=np.int16)
    recovered = np.array(recovered,dtype=np.int16)
    
    for w in range(width):
        toInverse = np.zeros(height, dtype=np.complex_)
        for h in range(height):
            magnitude = magnitudeVals[height-h-1,w]
            phase = phaseVals[height-h-1,w]
            toInverse[h] = magnitude * math.cos(phase) + (1j * magnitude * math.sin(phase))
        signal = np.fft.irfft(toInverse)
        recovered[w*WINDOW_STEP:w*WINDOW_STEP + WINDOW_LENGTH] += signal[:WINDOW_LENGTH].astype(np.int16)
    return recovered

In [7]:
rate = 16000

In [8]:
print("phaseMax {} phaseMin {} magnitudeMax {} magnitudeMin {}".format(phaseMax,phaseMin,magnitudeMax,magnitudeMin))

phaseMax 3.141592653589793 phaseMin -3.141592653589793 magnitudeMax 5432292.907520762 magnitudeMin 0.0


In [9]:
path_Voice = './Voice'

In [10]:
 # Imports
from scipy.io import wavfile
import scipy.signal as sps

# Your new sampling rate
new_rate = 16000

for filename in tqdm(glob.glob(os.path.join(path_Voice, '*.wav'))):
    sampling_rate, data = wavfile.read(filename)
    number_of_samples = round(len(data) * float(new_rate) / sampling_rate)
    data = sps.resample(data, number_of_samples)
    data = np.asarray(data, dtype=np.int16)
    wavfile.write(filename,new_rate,data)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 23.49it/s]


In [11]:
# Read All the lenght of wav file - Process it for 1 sec and queue up for prediction

In [12]:
# Implement

In [13]:
from pydub import AudioSegment
from pydub.utils import make_chunks

In [14]:
filename = glob.glob(os.path.join(path_Voice, '*.wav'))
print(filename)

['./Voice\\Voice 002.wav']


In [15]:
for filename in tqdm(glob.glob(os.path.join(path_Voice, '*.wav'))):
    myaudio = AudioSegment.from_file(filename) 
    chunk_length_ms = 1000 # pydub calculates in millisec
    chunks = make_chunks(myaudio, chunk_length_ms) #Make chunks of one sec
    
    for i, chunk in enumerate(chunks):
        
        chunk_name = "chunk{0}.wav".format(i)
        name = path_Voice+'/'+chunk_name
        
        chunk.export(name, format="wav")
        
        rate, data = wavfile.read(path_Voice+'/'+chunk_name)
        
        
        if len(data.shape) >= 2 and data.size > 0:
            if data.shape[-1] > 1:
                data = data.mean(axis=-1)
            else:
                data = np.reshape(data, data.shape[:-1])
        img = generateSpectrogramForWave(data)
        np.save(path_Voice+'/'+chunk_name[:-4]+'.npy', img) # save

  0%|                                                                                                                                                                                     | 0/1 [00:00<?, ?it/s]

0.16s
0.16s


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.27it/s]

0.11s


In [16]:
import tensorflow as tf
from tensorflow import keras

In [17]:
new_model = tf.keras.models.load_model('model.h5')

# Check its architecture
new_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 257, 62, 3)] 0                                            
__________________________________________________________________________________________________
block_1_conv (Conv2D)           (None, 257, 62, 64)  1792        input_2[0][0]                    
__________________________________________________________________________________________________
block_1_lrelu (LeakyReLU)       (None, 257, 62, 64)  0           block_1_conv[0][0]               
__________________________________________________________________________________________________
block_1_drop (Dropout)          (None, 257, 62, 64)  0           block_1_lrelu[0][0]              
____________________________________________________________________________________________

In [18]:
ROW = 257
COL = 62

test_pred = []

In [19]:
count = 0
rate = 16000

for filename in tqdm(glob.glob(os.path.join(path_Voice, '*.npy'))):
    img_test = np.load(filename)
    row_,col_,_ = img_test.shape
    
    if col_ < COL:
        break
    
    print(img_test.shape)
    
    img_test = img_test/255
    img_test = img_test.reshape(-1, ROW,COL,3)
    decoded_imgs = new_model.predict(img_test) #predict
    decoded_imgs = decoded_imgs.reshape(ROW,COL,3)
    decoded_imgs = decoded_imgs*255
    decoded_imgs = decoded_imgs.astype(np.int16)
    data = recoverSignalFromSpectrogram(decoded_imgs)
    file = './'+"testpred_{}".format(count)+'.wav'
    scipy.io.wavfile.write(file, rate, data)
    test_pred.append(file)
    count = count+1

  0%|                                                                                                                                                                                     | 0/3 [00:00<?, ?it/s]

(257, 62, 3)


 33%|█████████████████████████████████████████████████████████▋                                                                                                                   | 1/3 [00:02<00:04,  2.08s/it]

(257, 62, 3)


 67%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                         | 2/3 [00:03<00:01,  1.89s/it]


In [20]:
test_pred

['./testpred_0.wav', './testpred_1.wav']

In [21]:
from pydub import AudioSegment

In [22]:
sound = 0

In [23]:
path_recovered = './Voice/recovered/'

In [24]:
for i in range(len(test_pred)):
    print(test_pred[i])
    sound += AudioSegment.from_wav(test_pred[i])
    os.remove(test_pred[i])
sound.export(path_recovered+"soundJoined.wav", format="wav")

./testpred_0.wav
./testpred_1.wav


<_io.BufferedRandom name='./Voice/recovered/soundJoined.wav'>

In [25]:
def signaltonoise(a, axis, ddof): 
    a = np.asanyarray(a) 
    m = a.mean(axis) 
    sd = a.std(axis = axis, ddof = ddof) 
    return np.where(sd == 0, 0, m / sd)

In [26]:
import scipy.io.wavfile as wavfile
import numpy
import os.path
from scipy import stats


# def signaltonoise(a, axis=0, ddof=0):
#     a = np.asanyarray(a)
#     m = a.mean(axis)
#     sd = a.std(axis=axis, ddof=ddof)
#     return np.where(sd == 0, 0, m/sd)

# def signaltonoise(Arr, axis=0, ddof=0):
#     Arr = np.asanyarray(Arr)
#     me = Arr.mean(axis)
#     sd = Arr.std(axis=axis, ddof=ddof)
#     return np.where(sd == 0, 0, me/sd)

    
def signaltonoise(a, axis=0, ddof=0):
    mx = np.amax(a)
    a = np.divide(a,mx)
    a = np.square(a)
    a = np.asanyarray(a)
    m = a.mean(axis)
    sd = a.std(axis=axis, ddof=ddof)
    return np.where(sd == 0, 0, m/sd)


# def snr(file):
#   if (os.path.isfile(file)):
#     data = wavfile.read(file)[1]
#     singleChannel = data
#     try:
#       singleChannel = numpy.sum(data, axis=1)
#     except:
#       # was mono after all
#       pass
      
#     norm = singleChannel / (max(numpy.amax(singleChannel), -1 * numpy.amin(singleChannel)))
#     return stats.signaltonoise(norm)

In [27]:
clean = './Voice/results'
noisy = './Voice/samples'

In [28]:
for filename in tqdm(glob.glob(os.path.join(noisy, '*.wav'))):
    data = wavfile.read(filename)[1]
    
    #Single Channel
    if len(data.shape) >= 2 and data.size > 0:
        if data.shape[-1] > 1:
            data = data.mean(axis=-1)
        else:
            data = np.reshape(data, data.shape[:-1])
    snr = signaltonoise(data)
    snr_db = 10 * np.log10(snr) 
    print(filename,snr,snr_db)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 206.90it/s]

./Voice/samples\19-198-0034.wav 0.610921998799237 -2.140142359989448
./Voice/samples\196-122150-0032.wav 0.3914110236610155 -4.073669470793176
./Voice/samples\3982-182255-0009.wav 0.4539806918618618 -3.4296261761903213
./Voice/samples\4406-16883-0021.wav 0.4031533440378431 -3.945297335053095
./Voice/samples\6437-66173-0052.wav 0.4451663869406391 -3.5147763525671123
./Voice/samples\83-11691-0035.wav 0.3876813066572281 -4.115251394398863
./Voice/samples\8609-262281-0010.wav 0.39839244584157096 -3.9968890508876282
./Voice/samples\amb.wav 0.4071004999408123 -3.902983642838214
./Voice/samples\hairdryer.wav 0.5047302965775814 -2.969406258358389
./Voice/samples\Voice 002.wav 0.3450745918277261 -4.620870170494259
./Voice/samples\Voice 003.wav 0.37636136813976545 -4.243949613998555
./Voice/samples\Voice 004.wav 0.24028040323556604 -6.192816478965057
./Voice/samples\Voice 006.wav 0.24169029235172376 -6.167407929632485


In [29]:
for filename in tqdm(glob.glob(os.path.join(clean, '*.wav'))):
    data = wavfile.read(filename)[1]
    
    #Single Channel
    if len(data.shape) >= 2 and data.size > 0:
        if data.shape[-1] > 1:
            data = data.mean(axis=-1)
        else:
            data = np.reshape(data, data.shape[:-1])
            
    snr = signaltonoise(data)
    snr_db = 10 * np.log10(snr) 
    print(filename,snr,snr_db)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 501.34it/s]

./Voice/results\19-198-0034.wav 0.360540155898119 -4.430463578134739
./Voice/results\196-122150-0032.wav 0.3115271478101334 -5.0650410107415365
./Voice/results\3982-182255-0009.wav 0.3192070938789617 -4.959274656977031
./Voice/results\4406-16883-0021.wav 0.3768884543647751 -4.23787166563317
./Voice/results\6437-66173-0052.wav 0.35910947735905774 -4.44773133135056
./Voice/results\83-11691-0035.wav 0.3831126729598597 -4.166734817796328
./Voice/results\8609-262281-0010.wav 0.15567534530167088 -8.077801623019603
./Voice/results\amb.wav 0.2952368350016249 -5.29829459044242
./Voice/results\hairdryer.wav 0.26894718925215066 -5.70332990139004
./Voice/results\Voice 002.wav 0.2903214598370544 -5.371208610186352
./Voice/results\Voice 003.wav 0.28206661975600605 -5.496483059521439
./Voice/results\Voice 004.wav 0.2432314870795846 -6.139802049759371
./Voice/results\Voice 006.wav 0.23557507891819165 -6.278706547201235


# Remove ".PNG"

In [30]:
import os
from tqdm import tqdm
import glob
import shutil

for filename in glob.glob(os.path.join(clean, '*.png')):
    os.remove(filename)
for filename in glob.glob(os.path.join(noisy, '*.png')):
    os.remove(filename)

# Spectrogram of Clean Files

In [31]:
wav_list = os.listdir(clean)
num_wavs = len(clean)

In [32]:
for n in range(num_wavs-2):
    name_A = wav_list[n]
    path_A = os.path.join(clean,name_A)
    rate, data = wavfile.read(path_A)
    # Average over windows.
    if len(data.shape) >= 2 and data.size > 0:
        if data.shape[-1] > 1:
            data = data.mean(axis=-1)
        else:
            data = np.reshape(data, data.shape[:-1])
    # Only use a short clip for our demo
    if np.shape(data)[0] / float(rate) > 5:
        data = data[0 : rate * 5]
    img = generateSpectrogramForWave(data)
    img.save(path_A[:-4]+'.png')

0.88s
0.83s
0.84s
0.85s
0.82s
0.97s
0.83s
0.34s
0.30s
0.36s
0.31s
0.34s
0.37s


In [33]:
# Spectrogram of noisy Files

In [34]:
wav_list = os.listdir(noisy)
num_wavs = len(noisy)

In [35]:
for n in range(num_wavs-2):
    name_A = wav_list[n]
    path_A = os.path.join(noisy,name_A)
    rate, data = wavfile.read(path_A)
    # Average over windows.
    if len(data.shape) >= 2 and data.size > 0:
        if data.shape[-1] > 1:
            data = data.mean(axis=-1)
        else:
            data = np.reshape(data, data.shape[:-1])
    # Only use a short clip for our demo
    if np.shape(data)[0] / float(rate) > 5:
        data = data[0 : rate * 5]
    img = generateSpectrogramForWave(data)
    img.save(path_A[:-4]+'.png')

0.85s
0.80s
0.79s
0.81s
0.80s
0.85s
0.85s
0.43s
1.38s
0.45s
0.54s
0.46s
0.50s


In [36]:
# from snreval import SNREval

# snr_eval_clean = SNREval(clean)
# snr_eval_noisy = SNREval(noisy)

# # Either with a single .wav file
# # df = snr_eval.eval("/path/to/wave/file.wav")

# # Or with a multiple .wav files listed in a .txt file
# df_clean = snr_eval_clean.eval(clean+'/'+'results.txt')
# df_noisy = snr_eval_noisy.eval(noisy+'/'+'samples.txt')

# print(df_clean)

In [37]:
# Post Processing After Removing Noise From CNN
# https://www.ombarus.com/desnoise.py
# https://www.youtube.com/watch?v=f9P7SeUlzQg

In [38]:
# import sys
# import os
# os.environ["path"] = os.path.dirname(sys.executable) + ";" + os.environ["path"]
# import glob
# import operator
# import re
# import datetime
# import dateutil.relativedelta
# import win32gui
# import win32ui
# import win32con
# import win32api
# import numpy
# import json
# import csv
# import urllib.request
# import urllib.error
# import scipy.ndimage
# import scipy.stats
# import multiprocessing
# import subprocess
# import matplotlib.pyplot as plt
# from bs4 import BeautifulSoup
# from PIL import Image
# from time import strftime
# from time import sleep
# from operator import itemgetter
# from pydub import AudioSegment

In [39]:
###############################################################################
# GLOBAL CONSTANTS
###############################################################################

# TEST_FILE = "test.wav"
# SILENCE_FILE = "silence.wav"
# SILENCE_PROFILE = "silence.prof"
# SOX_PATH = '"C:\\Program Files (x86)\\sox-14-4-2\\sox.exe"'

# PRINT_LEVEL=2


###############################################################################
# UTILITIES
###############################################################################

# def myprint(str, level=0):
# 	if (level >= PRINT_LEVEL):
# 		print(str)
###############################################################################
# MAIN
###############################################################################
# def find_silence(in_file):
# 	sound1 = AudioSegment.from_file(in_file, format="wav")
# 	ms = 0
# 	current_silence = 0
# 	longuest_time = 500
# 	longuest_val = None
# 	for i in sound1:
# 		if i.dBFS > -38.0:
# 			length = ms - current_silence
# 			if length > longuest_time:
# 				longuest_val = sound1[current_silence : ms]
# 				longuest_time = length
# 			current_silence = ms + 1
# 		ms += 1
	
# 	myprint("longuest segment " + str(longuest_time / 1000.0) + " seconds", 2)
# 	longuest_val[200:-200].export(SILENCE_FILE, format="wav")

	
# def sox_denoise(in_file):
	
# 	#sox in.wav noiseprof out.prof
# 	#sox in.wav out.wav noisered out.prof 0.3
# 	sound1 = AudioSegment.from_file(SILENCE_FILE, format="wav")
# 	segment_length_sec = len(sound1) / 1000.0
# 	command = '{sox} "{wav_in}" -n trim 0 {silence_len} noiseprof {prof_out}'.format(sox=SOX_PATH, wav_in=SILENCE_FILE, silence_len=segment_length_sec, prof_out=SILENCE_PROFILE)
# 	myprint(command, 0)
# 	myprint(subprocess.call(command), 1)
	
# 	out_file = os.path.join(os.path.dirname(in_file), "cleaned", os.path.basename(in_file))
# 	if not os.path.exists(os.path.dirname(out_file)):
# 		os.makedirs(os.path.dirname(out_file))
	
# 	command = '{sox} "{wav_in}" "{wav_out}" noisered {prof_in} 0.3'.format(sox=SOX_PATH, wav_in=in_file, wav_out=out_file, prof_in=SILENCE_PROFILE)
# 	myprint(command, 0)
# 	myprint(subprocess.call(command), 1)
		
# def cleanup():
# 	os.remove(SILENCE_FILE)
# 	os.remove(SILENCE_PROFILE)
	
		
###############################################################################
# MAIN
###############################################################################
		
# def do_actions(actions, params):
# 	file_list = glob.glob(os.path.join(params["root"],"*.wav"))
# 	count = 1
# 	for f in file_list:
# 		myprint("[{}/{}] Processing : {}".format(count, len(file_list), f), 4)
# 		count += 1
# 		if "find_silence" in actions:
# 			find_silence(f)
# 		if "sox_denoise" in actions:
# 			sox_denoise(f)
# 	if "cleanup" in actions:
# 		cleanup()
		
		
# if __name__ == '__main__':
# 	actions = [
# 		"find_silence",
# 		"sox_denoise",
# 		"cleanup",
# 		"nothing" # just so I don't need to play with the last ,
# 	]
# 	params = {
# 		"root" : sys.argv[1],
# 		#"root" : ".",
# 		"nothing" : None # don't have to deal with last ,
# 	}
# 	do_actions(actions, params)
	
# 	sys.exit(0)